In [11]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import pyarrow as pa
import dask
import fsspec
from datetime import datetime, timedelta

import dask.dataframe as dd
from dask.distributed import Client
import os
import xarray as xr
import glob


In [12]:
def load_dataset(filename, engine="h5netcdf", *args, **kwargs) -> xr.Dataset:
    """Load a NetCDF dataset from local file system or cloud bucket."""
    with fsspec.open(filename, mode="rb") as file:
        dataset = xr.load_dataset(file, engine=engine, *args, **kwargs)
        print(dataset)
    return dataset

dir_files = [os.path.join("short_range_18files\input", files) for files in os.listdir("short_range_18files\input")]
print(dir_files)

['short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f001.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f002.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f003.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f004.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f005.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f006.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f007.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f008.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f009.conus.nc', 'short_range_18files\\input\\nwm.20220911_short_range_nwm.t00z.short_range.channe

In [13]:

ds = xr.concat([load_dataset(f) for f in dir_files],dim="time")
df = ds.to_dataframe()

df.to_parquet("data/multiple_files_short_range_18files.parquet"
             ,  engine="pyarrow"
             , compression="snappy"
             #, overwrite = True
             #, partition_on = 'feature_id'
             )

<xarray.Dataset>
Dimensions:         (time: 1, reference_time: 1, feature_id: 2776738)
Coordinates:
  * time            (time) datetime64[ns] 2022-09-11T01:00:00
  * reference_time  (reference_time) datetime64[ns] 2022-09-11
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
Data variables:
    crs             |S1 b''
    streamflow      (feature_id) float64 0.18 0.01 0.01 0.01 ... 0.0 0.0 0.0 0.0
    nudge           (feature_id) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    velocity        (feature_id) float64 0.08 0.1 0.1 0.01 ... 0.02 0.0 0.02
    qSfcLatRunoff   (feature_id) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qBucket         (feature_id) float64 0.00341 0.00858 ... 0.00013 0.00016
    qBtmVertRunoff  (feature_id) float64 12.18 26.58 9.311 ... 0.396 0.528 0.746
Attributes: (12/19)
    TITLE:                      OUTPUT FROM NWM v2.2
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=637000

In [15]:


ddf = dd.read_parquet(
    "data/multiple_files_short_range_18files.parquet"
    , storage_options={"anon": True}
    #, columns=["streamflow", "velocity"]
    #index="reference_time",

)

result = ddf.shape[0].compute()
result = ddf.compute()
#%time result
%time ddf.loc['2022-09-11 02:00'].compute()


CPU times: total: 16.4 s
Wall time: 16.4 s


crs  streamflow  nudge  \
time                reference_time feature_id                           
2022-09-11 02:00:00 2022-09-11     101         b''        0.18    0.0   
                                   179         b''        0.01    0.0   
                                   181         b''        0.01    0.0   
                                   183         b''        0.01    0.0   
                                   185         b''        0.01    0.0   
...                                            ...         ...    ...   
                                   1180001800  b''        0.00    0.0   
                                   1180001801  b''        0.00    0.0   
                                   1180001802  b''        0.00    0.0   
                                   1180001803  b''        0.00    0.0   
                                   1180001804  b''        0.00    0.0   

                                               velocity  qSfcLatRunoff  \
time                reference_time feature_id                            
2022-09-11 02:00:00 2022-09-11     101             0.08            0.0   
                                   179             0.10            0.0   
                                   181             0.10            0.0   
                                   183             0.01            0.0   
                                   185             0.01            0.0   
...                                                 ...            ...   
                                   1180001800      0.02            0.0   
                                   1180001801      0.01            0.0   
                                   1180001802      0.02            0.0   
                                   1180001803      0.00            0.0   
                                   1180001804      0.02            0.0   

                                               qBucket  qBtmVertRunoff  
time                reference_time feature_id                           
2022-09-11 02:00:00 2022-09-11     101         0.00341       12.171001  
                                   179         0.00857       26.513001  
                                   181         0.00234        9.282000  
                                   183         0.00087        3.371000  
                                   185         0.00033        1.288000  
...                                                ...             ...  
                                   1180001800  0.00000        0.005000  
                                   1180001801  0.00003        0.214000  
                                   1180001802  0.00010        0.397000  
                                   1180001803  0.00013        0.531000  
                                   1180001804  0.00016        0.758000  

[2776738 rows x 7 columns]